In [1]:
"""
A script that generates training_data directory using the given csv file.
@author Isu Kim, gooday2die
"""

'\nA script that generates training_data directory using the given csv file.\n@author Isu Kim, gooday2die\n'

In [2]:
import pandas as pd
import os

In [3]:
def get_noise_list(labels):
    """
    A function that generates a list of tuples that represents the start and end of noises.
    :param labels: the string that represents labels
    :return: a list of tuples that contain (start, end) of each noises.
    """
    starts = list()
    ends = list()
    
    for i in labels:
        starts.append(i['start'])
        ends.append(i['end'])
    
    noises = list()
    noises.append((0, starts[0])) # The start of file to starts[0] is considered a noise

    for i in range(len(ends)):
        try:
            noises.append((ends[i], starts[i + 1]))
        except IndexError:  # If index error, that means there is no more noise left
            pass

    noises.append((ends[len(ends) - 1], -1))  # The last end to the end of the file is considered a noise
    return noises

In [4]:
def generate_noise_df(df):
    """
    A function that generates a pandas.DataFrame that contains all start and ends of noises.
    :param df: a pandas.DataFrame object that represents the original dataframe
    :return: return a DataFrame object that includes all noise starts and ends.
    """
    file_names = list()
    start = list()
    end = list()
    class_name = list()
    
    for i in range(len(df)):
        cur_df = df.loc[i]
        noise_list = get_noise_list(eval(cur_df['labels']))
        cur_file_name = os.path.join(os.getcwd(), cur_df['audio'][1:])
        
        for noise in noise_list:
            start.append(noise[0])
            end.append(noise[1])
            class_name.append("noise")
            file_names.append(cur_file_name)

    modified_df = pd.DataFrame()
    modified_df['file_name'] = file_names
    modified_df['start'] = start
    modified_df['end'] = end
    modified_df['class'] = class_name
    
    return modified_df

In [5]:
def modify_df(file_name):
    """
    A function that reads file in csv format which was exported by LabelStudio.
    This function will transform given csv into dataframe that is more suitable for our sound classification.
    :param file_name: the string object that represents file name
    :returns: pandas.DataFrame object that represents data from the csv turned into our style.
    """
    data = pd.read_csv(file_name)
    
    file_names = list()
    start = list()
    end = list()
    class_name = list()

    for i in range(len(data)):
        instance = data.loc[i]
        cur_file_name = os.path.join(os.getcwd(), instance['audio'][1:])
        cur_class = eval(instance['intent'])  # Each values are recorded as str, so need eval
        cur_time_lines = eval(data.loc[i]['labels'])

        for j in range(len(cur_class)):
            cur_start = float(cur_time_lines[j]['start'])
            cur_end = float(cur_time_lines[j]['end'])

            start.append(cur_start)
            end.append(cur_end)
            class_name.append(cur_class[j])
            file_names.append(cur_file_name)

    modified_df = pd.DataFrame()
    modified_df['file_name'] = file_names
    modified_df['start'] = start
    modified_df['end'] = end
    modified_df['class'] = class_name
    
    noise_df = generate_noise_df(data)
    modified_df = modified_df.append(noise_df, ignore_index=True)
    
    return modified_df

In [6]:
from PIL import Image
from pydub import AudioSegment
import sqlite3
import string
import os

def generate_db():
    """
    A function that generates training_data/data.db file and initializes the table values.
    :return: None
    """
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    conn.execute("CREATE TABLE data(data_name TEXT, data_count INTEGER)")
    
    default_data = [(x, 0) for x in string.ascii_lowercase]
    default_data = default_data + [(str(x), 0) for x in range(0, 10, 1)]
    default_data.append(("noise", 0))
    
    cur.executemany("INSERT INTO data VALUES(?, ?)", default_data)
    
    conn.commit()
    conn.close()
    
def get_data_count(data_name):
    """
    A function that gets data count from data.db for specific data_name
    :param data_name: the data_name to look for data counts
    :return: returns count of data_count value for provided data_name, returns -1 if data_name does not exist.
    """
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    cur.execute("SELECT data_count FROM data WHERE data_name=\"" + data_name + "\"")
    rows = cur.fetchall()
    try:
        return rows[0][0]
    except IndexError:
        return -1
    
    conn.close()
    
def update_data_count(data_name):
    """
    A function that adds 1 to the current data_count for specified data_name
    :param data_name: the data_name to update values into
    :return: None
    """
    before_count = get_data_count(data_name)    
    new_count = before_count + 1
    
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    cur.execute("UPDATE data SET data_count=" + str(new_count) + " WHERE data_name=\"" + data_name + "\"")
    conn.commit()
    conn.close()

def generate_all_directories():
    """
    A function that generates training_data directories.
    This will generate trainig_data/a~z and 0~9 directories.
    :return: None
    """
    dir_names = [x for x in string.ascii_lowercase]
    dir_names = dir_names + [str(x) for x in range(0, 10, 1)]
    dir_names.append("noise")


    training_data_path = os.path.join(os.getcwd(), "training_data")

    if not os.path.exists(training_data_path):  # generate training_data directory
        os.mkdir(training_data_path)
    
    for i in dir_names:  # generate subdirectories such as a ~ z, 0 ~ 9
        new_dir= os.path.join(training_data_path, i)
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

def process_df(df):
    """
    A function that splits each sound classifications from dataframe into different files
    :param df: the pandas.DataFrame object that represents single row of information.
    """
    file_name = df['file_name']
    new_audio = AudioSegment.from_wav(file_name)
    
    start = df['start'] * 1000
    
    if df['end'] == -1:
        end = len(new_audio)
    else:
        end = df['end'] * 1000
    class_name = df['class']
    
    tmp_audio = new_audio[start:end]
    
    save_dir = os.path.join(os.getcwd(), "training_data", class_name)
    new_file_name = str(get_data_count(class_name)) + ".wav"
    new_file_name = os.path.join(save_dir, new_file_name)
    update_data_count(class_name)
    tmp_audio.export(new_file_name, format="wav")

/home/gooday2die/.local/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [7]:
df = modify_df("./data/project-3-at-2022-06-24-09-35-bc8bde98.csv")

/tmp/ipykernel_106797/3325555645.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  modified_df = modified_df.append(noise_df, ignore_index=True)


In [9]:
try:
    generate_db()
    print("[+] DB not found, generating a new one")
except sqlite3.OperationalError:  # when db already exists
    pass
generate_all_directories()

for i in range(len(df)):
    print("[+] Processing file " + str(i + 1) + " / " + str(len(df)))
    process_df(df.loc[i])

from IPython.display import Audio 
#Audio("/home/gooday2die/projects/Anti-Captcha/sounddooo/sounds/training_data/j/0.wav", autoplay=True)

[+] DB not found, generating a new one
[+] Processing file 1 / 2187
[+] Processing file 2 / 2187
[+] Processing file 3 / 2187
[+] Processing file 4 / 2187
[+] Processing file 5 / 2187
[+] Processing file 6 / 2187
[+] Processing file 7 / 2187
[+] Processing file 8 / 2187
[+] Processing file 9 / 2187
[+] Processing file 10 / 2187
[+] Processing file 11 / 2187
[+] Processing file 12 / 2187
[+] Processing file 13 / 2187
[+] Processing file 14 / 2187
[+] Processing file 15 / 2187
[+] Processing file 16 / 2187
[+] Processing file 17 / 2187
[+] Processing file 18 / 2187
[+] Processing file 19 / 2187
[+] Processing file 20 / 2187
[+] Processing file 21 / 2187
[+] Processing file 22 / 2187
[+] Processing file 23 / 2187
[+] Processing file 24 / 2187
[+] Processing file 25 / 2187
[+] Processing file 26 / 2187
[+] Processing file 27 / 2187
[+] Processing file 28 / 2187
[+] Processing file 29 / 2187
[+] Processing file 30 / 2187
[+] Processing file 31 / 2187
[+] Processing file 32 / 2187
[+] Proces

[+] Processing file 293 / 2187
[+] Processing file 294 / 2187
[+] Processing file 295 / 2187
[+] Processing file 296 / 2187
[+] Processing file 297 / 2187
[+] Processing file 298 / 2187
[+] Processing file 299 / 2187
[+] Processing file 300 / 2187
[+] Processing file 301 / 2187
[+] Processing file 302 / 2187
[+] Processing file 303 / 2187
[+] Processing file 304 / 2187
[+] Processing file 305 / 2187
[+] Processing file 306 / 2187
[+] Processing file 307 / 2187
[+] Processing file 308 / 2187
[+] Processing file 309 / 2187
[+] Processing file 310 / 2187
[+] Processing file 311 / 2187
[+] Processing file 312 / 2187
[+] Processing file 313 / 2187
[+] Processing file 314 / 2187
[+] Processing file 315 / 2187
[+] Processing file 316 / 2187
[+] Processing file 317 / 2187
[+] Processing file 318 / 2187
[+] Processing file 319 / 2187
[+] Processing file 320 / 2187
[+] Processing file 321 / 2187
[+] Processing file 322 / 2187
[+] Processing file 323 / 2187
[+] Processing file 324 / 2187
[+] Proc

[+] Processing file 585 / 2187
[+] Processing file 586 / 2187
[+] Processing file 587 / 2187
[+] Processing file 588 / 2187
[+] Processing file 589 / 2187
[+] Processing file 590 / 2187
[+] Processing file 591 / 2187
[+] Processing file 592 / 2187
[+] Processing file 593 / 2187
[+] Processing file 594 / 2187
[+] Processing file 595 / 2187
[+] Processing file 596 / 2187
[+] Processing file 597 / 2187
[+] Processing file 598 / 2187
[+] Processing file 599 / 2187
[+] Processing file 600 / 2187
[+] Processing file 601 / 2187
[+] Processing file 602 / 2187
[+] Processing file 603 / 2187
[+] Processing file 604 / 2187
[+] Processing file 605 / 2187
[+] Processing file 606 / 2187
[+] Processing file 607 / 2187
[+] Processing file 608 / 2187
[+] Processing file 609 / 2187
[+] Processing file 610 / 2187
[+] Processing file 611 / 2187
[+] Processing file 612 / 2187
[+] Processing file 613 / 2187
[+] Processing file 614 / 2187
[+] Processing file 615 / 2187
[+] Processing file 616 / 2187
[+] Proc

[+] Processing file 875 / 2187
[+] Processing file 876 / 2187
[+] Processing file 877 / 2187
[+] Processing file 878 / 2187
[+] Processing file 879 / 2187
[+] Processing file 880 / 2187
[+] Processing file 881 / 2187
[+] Processing file 882 / 2187
[+] Processing file 883 / 2187
[+] Processing file 884 / 2187
[+] Processing file 885 / 2187
[+] Processing file 886 / 2187
[+] Processing file 887 / 2187
[+] Processing file 888 / 2187
[+] Processing file 889 / 2187
[+] Processing file 890 / 2187
[+] Processing file 891 / 2187
[+] Processing file 892 / 2187
[+] Processing file 893 / 2187
[+] Processing file 894 / 2187
[+] Processing file 895 / 2187
[+] Processing file 896 / 2187
[+] Processing file 897 / 2187
[+] Processing file 898 / 2187
[+] Processing file 899 / 2187
[+] Processing file 900 / 2187
[+] Processing file 901 / 2187
[+] Processing file 902 / 2187
[+] Processing file 903 / 2187
[+] Processing file 904 / 2187
[+] Processing file 905 / 2187
[+] Processing file 906 / 2187
[+] Proc

[+] Processing file 1137 / 2187
[+] Processing file 1138 / 2187
[+] Processing file 1139 / 2187
[+] Processing file 1140 / 2187
[+] Processing file 1141 / 2187
[+] Processing file 1142 / 2187
[+] Processing file 1143 / 2187
[+] Processing file 1144 / 2187
[+] Processing file 1145 / 2187
[+] Processing file 1146 / 2187
[+] Processing file 1147 / 2187
[+] Processing file 1148 / 2187
[+] Processing file 1149 / 2187
[+] Processing file 1150 / 2187
[+] Processing file 1151 / 2187
[+] Processing file 1152 / 2187
[+] Processing file 1153 / 2187
[+] Processing file 1154 / 2187
[+] Processing file 1155 / 2187
[+] Processing file 1156 / 2187
[+] Processing file 1157 / 2187
[+] Processing file 1158 / 2187
[+] Processing file 1159 / 2187
[+] Processing file 1160 / 2187
[+] Processing file 1161 / 2187
[+] Processing file 1162 / 2187
[+] Processing file 1163 / 2187
[+] Processing file 1164 / 2187
[+] Processing file 1165 / 2187
[+] Processing file 1166 / 2187
[+] Processing file 1167 / 2187
[+] Proc

[+] Processing file 1404 / 2187
[+] Processing file 1405 / 2187
[+] Processing file 1406 / 2187
[+] Processing file 1407 / 2187
[+] Processing file 1408 / 2187
[+] Processing file 1409 / 2187
[+] Processing file 1410 / 2187
[+] Processing file 1411 / 2187
[+] Processing file 1412 / 2187
[+] Processing file 1413 / 2187
[+] Processing file 1414 / 2187
[+] Processing file 1415 / 2187
[+] Processing file 1416 / 2187
[+] Processing file 1417 / 2187
[+] Processing file 1418 / 2187
[+] Processing file 1419 / 2187
[+] Processing file 1420 / 2187
[+] Processing file 1421 / 2187
[+] Processing file 1422 / 2187
[+] Processing file 1423 / 2187
[+] Processing file 1424 / 2187
[+] Processing file 1425 / 2187
[+] Processing file 1426 / 2187
[+] Processing file 1427 / 2187
[+] Processing file 1428 / 2187
[+] Processing file 1429 / 2187
[+] Processing file 1430 / 2187
[+] Processing file 1431 / 2187
[+] Processing file 1432 / 2187
[+] Processing file 1433 / 2187
[+] Processing file 1434 / 2187
[+] Proc

[+] Processing file 1673 / 2187
[+] Processing file 1674 / 2187
[+] Processing file 1675 / 2187
[+] Processing file 1676 / 2187
[+] Processing file 1677 / 2187
[+] Processing file 1678 / 2187
[+] Processing file 1679 / 2187
[+] Processing file 1680 / 2187
[+] Processing file 1681 / 2187
[+] Processing file 1682 / 2187
[+] Processing file 1683 / 2187
[+] Processing file 1684 / 2187
[+] Processing file 1685 / 2187
[+] Processing file 1686 / 2187
[+] Processing file 1687 / 2187
[+] Processing file 1688 / 2187
[+] Processing file 1689 / 2187
[+] Processing file 1690 / 2187
[+] Processing file 1691 / 2187
[+] Processing file 1692 / 2187
[+] Processing file 1693 / 2187
[+] Processing file 1694 / 2187
[+] Processing file 1695 / 2187
[+] Processing file 1696 / 2187
[+] Processing file 1697 / 2187
[+] Processing file 1698 / 2187
[+] Processing file 1699 / 2187
[+] Processing file 1700 / 2187
[+] Processing file 1701 / 2187
[+] Processing file 1702 / 2187
[+] Processing file 1703 / 2187
[+] Proc

[+] Processing file 1942 / 2187
[+] Processing file 1943 / 2187
[+] Processing file 1944 / 2187
[+] Processing file 1945 / 2187
[+] Processing file 1946 / 2187
[+] Processing file 1947 / 2187
[+] Processing file 1948 / 2187
[+] Processing file 1949 / 2187
[+] Processing file 1950 / 2187
[+] Processing file 1951 / 2187
[+] Processing file 1952 / 2187
[+] Processing file 1953 / 2187
[+] Processing file 1954 / 2187
[+] Processing file 1955 / 2187
[+] Processing file 1956 / 2187
[+] Processing file 1957 / 2187
[+] Processing file 1958 / 2187
[+] Processing file 1959 / 2187
[+] Processing file 1960 / 2187
[+] Processing file 1961 / 2187
[+] Processing file 1962 / 2187
[+] Processing file 1963 / 2187
[+] Processing file 1964 / 2187
[+] Processing file 1965 / 2187
[+] Processing file 1966 / 2187
[+] Processing file 1967 / 2187
[+] Processing file 1968 / 2187
[+] Processing file 1969 / 2187
[+] Processing file 1970 / 2187
[+] Processing file 1971 / 2187
[+] Processing file 1972 / 2187
[+] Proc